In [1]:
from transformers import (RobertaConfig,
                          RobertaForSequenceClassification,
                          RobertaTokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup)
import torch
import pytorch_lightning as pl
from torch.optim import lr_scheduler
from torch import nn
import numpy as np

In [2]:
roberta = RobertaForSequenceClassification.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [3]:
import Refactored_Baseline
import importlib
importlib.reload(Refactored_Baseline)

<module 'Refactored_Baseline' from '/Users/nissani/Desktop/moral_stories/Refactored_Baseline.py'>

In [4]:
model = Refactored_Baseline.Baseline_Model(roberta, lr = .01)

In [5]:
from test_load_data import *

#moral_processor = utils.MoralStoriesProcessor()
print(DatasetType.TRAIN.value)
#the_dataset = MoralStoryDataset("/Users/garbar/Downloads/moral_stories_datasets/classification/action+context/lexical_bias","train")
#training_data = moral_processor.get_train_examples("/Users/garbar/Downloads/moral_stories_datasets/classification/action+context/lexical_bias")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
path = "/Users/nissani/Desktop/moral_stories_datasets/classification/action+context/lexical_bias"
datatype = DatasetType.TEST
modeltype = ModelNames.ROBERTA
tasktype = TaskTypes.ACTION_CONTEXT_CLS
the_moral_story = MoralStoryClassifyDataset(path,dataset_type = DatasetType.TEST,tokenizer=tokenizer,model_name=modeltype,tasktype=tasktype)

temp = (the_moral_story._process_data(the_moral_story._moral_story_data[0:10]))
print("OK")

train
OK


In [6]:
the_moral_story.process_data(start_index = 0, stop_index = 28)

In [7]:
some_value = next(iter(the_moral_story))

In [8]:
len(some_value['attention_mask'])

72

In [9]:
print(the_moral_story._tokenized_data[1].input_ids)

[3, 243, 18, 1099, 7, 28, 350, 7337, 8, 3874, 82, 62, 4, 34057, 13356, 62, 182, 419, 42, 662, 4, 894, 1072, 7, 146, 103, 7080, 4, 3, 3, 34057, 117, 354, 5846, 14225, 29, 10, 319, 9, 26520, 8, 31592, 77, 442, 39, 7080, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
print(some_value['input_ids'][0])

tensor(3)


In [11]:
from torch.utils.data import random_split, DataLoader
data_loader = DataLoader(the_moral_story, batch_size=1, shuffle=False,num_workers=4)

In [12]:
trainer = pl.Trainer(min_epochs = 1, max_epochs = 2, checkpoint_callback = False)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [13]:
trainer.fit(model, data_loader)


  | Name  | Type                             | Params
-----------------------------------------------------------
0 | model | RobertaForSequenceClassification | 125 M 
-----------------------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.951   Total estimated model params size (MB)


/Users/nissani/Desktop/ms_venv/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [14]:
model.forward(next(iter(data_loader)))

(tensor(1.4454, grad_fn=<NllLossBackward>),
 tensor([[ 0.3065, -0.8702]], grad_fn=<AddmmBackward>))

In [15]:
model.training_step(next(iter(data_loader)), 1)

{'loss': tensor(2.3418, grad_fn=<NllLossBackward>),
 'preds': tensor([[ 0.8468, -1.3939]], grad_fn=<AddmmBackward>),
 'target': tensor([1]),
 'text': ''}